In [1]:
import json
import openai
import os
import re
from langchain_openai import ChatOpenAI
import transformers
from datasets import Dataset

In [14]:
file_path = 'C:\\Users\\PC\\Desktop\\DoYoung\\DS\\github\\bitamin_auto_readme_generator\\data\\object_detection\\output\\ocr_samples_txt\\netflix_text.txt'

with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

In [15]:
api_key_filepath = "C:\\Users\\PC\\Desktop\\DoYoung\\DS\\비타민NLP_240701\\text_summarization\\openai_api_key.json"
with open(api_key_filepath, 'r') as f:
    api_key = json.load(f)
api_key = api_key['OPENAI_API_KEY']

In [16]:
os.environ['OPENAI_API_KEY'] = api_key

In [17]:
llm_3 = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

In [18]:
llm_4 = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

### STEP1: 첫 5페이지의 텍스트만 추출

In [19]:
end_marker = text.find('<p.6>')

if end_marker != -1:
    text5 = text[:end_marker]
else:
    text5 = text

### STEP2: 마크다운 형식으로 팀원, 주제, main topic 추출

In [20]:
def extract_information(text5):
    prompt = f"""
    Extract the following information from the provided text:
    1. Name (excluding the team name, include all people listed)
    2. Title (include the entire title as it appears in the text)
    3. Main Topics (only main topics from the Table of Contents, excluding subtopics)

    The Main Topics should be extracted from the section that follows headers such as 'TABLE OF CONTENTS', '목차 소개', or any similar variation.
    Ensure that team member's name are not split into multiple lines. Name should be connected by commas if there are more than one.
    Ensure that title is not cut off and is extracted in their entirety.
    Exclude any subtopics or secondary information while extracting main topics.
    Stop at the next page marker '<p.'.

    Do not include any additional notes or explanations in the output.

    Text: {text5}


    Format the extracted information as follows:
    <subject>title</subject>
    <team>team members</team>
    <index>main topics</index>

    """

    response = llm_4.invoke(prompt)
    extracted_info = response.content.strip()
    return extracted_info

extracted_info = extract_information(text5)
print(extracted_info)


<subject>Netflix Stock Price Prediction with News Topic & Sentiment</subject>
<team>송규헌, 권도영, 이태경, 김서윤, 한진솔</team>
<index>INTRODUCTION, DATA PREPROCESSING, MODELING, CONCLUSIONS AND LIMITATIONS</index>


In [21]:
main_topics_match = re.search(r'<index>(.*?)</index>', extracted_info, re.DOTALL)
if main_topics_match:
    main_topics = main_topics_match.group(1).strip()
else:
    main_topics = ""

### STEP3: 전체 텍스트에서 대주제, 소주제, 핵심내용 추출

In [22]:
def extract_detailed_information(text, main_topics):
    prompt = f"""
    Extract detailed information for each main topic and its subtopics from the provided text.

    Format the extracted information as follows:
    <main>main topic</main>
    <sub>subtopic</sub>
    <content>content</content>

    Use the main topics provided below:
    {main_topics}
    
    The text is divided into pages using the format <p.number>. For example, page 2 is marked as <p.2>. 
    Make sure to extract text from all pages except for the first five pages to ensure no information is missed.
    If any part of the text seems to be related to the main topics but is not included in the main topics list(main_topics), include it as a subtopic under the appropriate main topic.
    Do not include any additional notes or explanations in the output.

    Text: {text}
    """

    response = llm_4.invoke(prompt)
    extracted_details = response.content.strip()
    return extracted_details

In [23]:
detailed_info = extract_detailed_information(text, main_topics)
print(detailed_info)

<main>INTRODUCTION</main>
<sub>Background of topic selection</sub>
<content>
1. 뉴스가 주가 변동에 미치는 영향 탐구 
   - 주가 예측에 뉴스를 활용할 수 있는지 탐구해보고자 함
   - 뉴스 감성분석 및 토픽 모델링 결과를 사용하여 예측
   - 뉴스 기사에서는 주로 한 기업에 대해 보도하고 있어 예측 대상은 한 개의 주식 종목으로 정함
   - 뉴스 데이터를 직접 활용하여 주가를 예측하는 프로젝트는 많지 않아 직접 뉴스 데이터를 활용하고자 함
2. 장기적인 추세를 고려할 수 있는 주가 예측 모델 구현
   - 기존 프로젝트는 주로 예측 일수나 시퀀스 길이를 1~5로 설정함
   - 단기적인 변동뿐만 아니라 장기적인 추세를 고려한 예측 결과를 얻고자 함
   - 주가 예측 그래프가 실제 주가 그래프를 단순 평행 이동한 형태로 나타나는 문제를 해결하고자 함
</content>
<sub>Brief Project Introduction</sub>
<content>
   - [목표] 뉴스 데이터를 활용한 주가 예측 모델의 최적화 및 효율적인 파라미터 선정
   - [주요 내용] 장기적인 추세 예측에 효과적인 모델 판별: LSTM VS GRU Vs Transformer
   - 실험을 통한 모델 파라미터 튜닝
   - 뉴스 감성분석과 토픽모델링의 주가 예측 유용성 검토
   - 각 실험 결과 평가에 필요한 지표 선정
   - [개발 환경] Colab
</content>
<sub>Data collection</sub>
<content>
1. 주가 데이터
   - FinanceDataReader 라이브러리를 활용하여 NETFLIX 기업의 2018년 1월 2일 ~ 2023년 12월 29일 주가 데이터 수집
   - 동일한 기간에 대한 핀터레스트, 메타플렛폼스, 스포티파이의 주가 데이터 수집
2. 뉴스 데이터
   - Stock News Ap를 활용하여 2018년 1월 2일부터 2023년 12월 29일 기간에

### STEP4: 요약

In [24]:
def summarize_content(content):
    if not content.strip():
        return content

    prompt = f"""
    다음 내용을 요약해줘:

    내용: {content}

    위 내용을 간결하게 요약해줘.
    """

    response = llm_4.invoke(prompt)
    summary = response.content.strip()
    return summary

summarized_detailed_info = re.sub(
    r'(<content>)(.*?)(</content>)',
    lambda m: f"{m.group(1)}{summarize_content(m.group(2))}{m.group(3)}",
    detailed_info,
    flags=re.DOTALL
)

print(summarized_detailed_info)

<main>INTRODUCTION</main>
<sub>Background of topic selection</sub>
<content>이 프로젝트는 뉴스가 주가 변동에 미치는 영향을 탐구하고, 이를 활용해 주가를 예측하는 것을 목표로 한다. 뉴스 감성분석과 토픽 모델링을 통해 특정 기업의 주가를 예측하며, 기존 프로젝트보다 더 긴 예측 기간을 고려해 장기적인 추세도 반영하고자 한다. 또한, 단순한 평행 이동 형태의 예측 문제를 해결하고자 한다.</content>
<sub>Brief Project Introduction</sub>
<content>목표는 뉴스 데이터를 활용해 주가 예측 모델을 최적화하고 효율적인 파라미터를 선정하는 것이다. 주요 내용은 LSTM, GRU, Transformer 모델의 장기 추세 예측 성능을 비교하고, 실험을 통해 모델 파라미터를 튜닝하는 것이다. 또한 뉴스 감성분석과 토픽모델링이 주가 예측에 얼마나 유용한지 검토하며, 실험 결과 평가에 필요한 지표를 선정한다. 개발 환경으로는 Colab을 사용한다.</content>
<sub>Data collection</sub>
<content>1. 주가 데이터: 
   - FinanceDataReader 라이브러리를 사용해 2018년 1월 2일부터 2023년 12월 29일까지 NETFLIX, 핀터레스트, 메타플렛폼스, 스포티파이의 주가 데이터를 수집.

2. 뉴스 데이터:
   - Stock News Ap를 통해 같은 기간 동안 NETFLIX 관련 뉴스 데이터를 수집.
   - 웹 스크래핑 후 FinBERT와 BERTopic을 활용해 감성 분석과 토픽 모델링을 시도.
   - 시계열 예측과 데이터셋의 정확도를 높이기 위해 Ap를 사용.</content>

<main>DATA PREPROCESSING</main>
<sub>Make derived Variable</sub>
<content>변화율(Roc), 이동 평균(MA), 단순 이동 평균(SMA), 12OMA 등의 파생변수를 생성할 때, 이전